In [ ]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

Disponible en: [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris)

| **Property**        | **Value**       |
|---------------------|-----------------|
| Classes             | 3               |
| Samples per class   | 50              |
| Samples total       | 150             |
| Dimensionality      | 4               |
| Features            | real, positive  |


| Tipo de flor | Número |
| ------------ | ------ |
| Setosa       | 0      |
| Versicolor   | 1      |
| Virginica    | 2      |

#### ¿qué observamos?

<details>
  <summary>👁️‍🗨️</summary>
  
  * Clase 0 (color morado oscuro) está bien separada: se agrupa con sépalos más cortos y más anchos.

  * Clase 1 y Clase 2 (verde y amarillo) están más superpuestas, especialmente en la región de sépalos largos y estrechos.

  * Esto sugiere que, al menos con las variables ``sepal length`` y ``sepal width``, la clase 0 se puede clasificar con mayor precisión que las otras dos.
</details>

### Modelo de clasificación

**Pregunta:**

_¿Cuáles modelos de clasificación podríamos usar para resolver este problema?_

## Cómo usar una distribución conjunta para clasificar

Anteriormente vimos que un modelo de clasificación (como árboles de decisión) presenta un _score_ del 73%.

Podemos adoptar un **enfoque probabilístico** para modelar la situación a partir de los datos disponibles.

En particular, podemos **estimar la distribución de probabilidad conjunta** de las variables de interés: la longitud del tallo $(L)$, el ancho del tallo $(W)$ y el tipo de flor $(T)$,

$$
P(L, W, T)
$$

utilizando los datos observados. Esta distribución nos permitirá **responder preguntas basadas en evidencia**, como predecir el tipo de flor más probable dado que observamos una cierta longitud y ancho del tallo.

A continuación, estimamos la **distribución conjunta empírica** a partir del conjunto de entrenamiento:

Primero, veámos en la tabla siguiente que las columnas (`L`, `W` `T`) representan todas las posibles combinaciones de valores de cada variable.

La tabla muestra la **distribución conjunta** de tres variables (`L`, `W`, `T`).

Usar `group.by().size()` nos permite *contar ocurrencias* para cada combinación.

Tenemos $4 * 3 * 3 = 36$ posibles combinaciones:

**NOTA:** no todas las combinaciones posibles de (`L`, `W` `T`) aparecen en los datos. `groupby().size()` solo muestra las combinaciones que realmente están presentes en el DataFrame.

Las combinaciones que no ocurren ni una sola vez, no aparecen en la salida porque su conteo es $0$.

| L | W | T | Frecuencia |
|---|---|---|------------|
| 4 | 2 | 0 | 2          |
| 4 | 2 | 1 | 1          |
| 4 | 2 | 2 | 1          |
| 4 | 3 | 0 | 16         |
| 5 | 2 | 1 | 17         |
| 5 | 2 | 2 | 5          |
| 5 | 3 | 0 | 22         |
| 5 | 3 | 1 | 6          |
| 5 | 3 | 2 | 1          |
| 6 | 2 | 1 | 10         |
| 6 | 2 | 2 | 11         |
| 6 | 3 | 1 | 9          |
| 6 | 3 | 2 | 18         |
| 7 | 2 | 2 | 3          |
| 7 | 3 | 1 | 1          |
| 7 | 3 | 2 | 8          |


Una vez que disponemos del conjunto de datos de entrenamiento, podemos estimar la 
**distribución conjunta empírica** de las variables discretas $L$, $W$ y $T$ como:

$$
\hat{P}(L = \ell, W = w, T = t) = \frac{\#\{i : (L_i, W_i, T_i) = (\ell, w, t)\}}{N}
$$

donde:
- $N$ es el número total de observaciones en el conjunto de entrenamiento,
- $(L_i, W_i, T_i)$ representa la $i$-ésima observación en los datos,
- y el numerador cuenta cuántas veces aparece la combinación exacta $(\ell, w, t)$.

Esta fórmula convierte las frecuencias observadas en probabilidades relativas, de forma que:

$$
\sum_{\ell} \sum_{w} \sum_{t} \hat{P}(L = \ell, W = w, T = t) = 1
$$

lo cual garantiza que $\hat{P}$ es una distribución de probabilidad válida.

Una vez que contamos con la distribución conjunta $\hat{P}(L, W, T)$, podemos construir un clasificador probabilístico de la siguiente manera:

1. Dado un par de valores observados de longitud y ancho del tallo, $(L = \ell, W = w)$, podemos calcular la distribución condicional del tipo de flor:

$$
P(T = t \mid L = \ell, W = w).
$$

2. Esta distribución nos da una probabilidad para cada posible valor de $T$ ($T = 0, 1, 2$). Como queremos predecir el tipo de flor más probable, elegimos el valor de $t$ que maximiza esta probabilidad:

$$
\hat{t} = \arg\max_{t} P(T = t \mid L = \ell, W = w).
$$

3. Este procedimiento se conoce como el **clasificador MAP (Maximum A Posteriori)**. Además, no necesitamos calcular explícitamente la distribución condicional, ya que:

$$
P(T = t \mid L = \ell, W = w) = \frac{P(T = t, L = \ell, W = w)}{P(L = \ell, W = w)}.
$$

Como el denominador $P(L = \ell, W = w)$ no depende de $t$, se puede omitir al hacer la maximización. Por lo tanto, el clasificador se simplifica a:

$$
\hat{t} = \arg\max_{t} P(T = t, L = \ell, W = w).
$$

Este enfoque nos permite clasificar directamente utilizando la distribución conjunta empírica que estimamos previamente.

$$
\hat{t} = \arg\max_{t} \, \hat{P}(T = t, L = 5, W = 3)
$$